In [1]:
# Confirm the number of cigars vs samtools mapped reads value (987)

def printNumMapped(filename):

    inputfile = open(filename, "r")


    unmapped_count = 0
    mapped_count = 0
    cs_count = 0

    for line in inputfile:

        # Header lines start with '@'
        if line[0] is "@":
            continue

        fields = line.split()

        cigar = fields[5]


        if fields[5] is "*": 
            unmapped_count += 1
        else: 
            mapped_count += 1
            #if len(fields) > 20:
                #print (fields[20])

        i = 10
        while i < len(fields):
            entry = fields[i]
            if "cs:Z:" in entry: 
                cs_count += 1
                #print (entry)
            i += 1

    print (filename)
    print (unmapped_count)
    print (mapped_count)
    print (cs_count)
    print ("")
    
    return mapped_count

In [2]:
import re

def count_penalties(cs_string):
    mismatches = 0
    insertions = 0
    deletions = 0
    matches = 0
    
    fields = re.split('(\W)', cs_string)
    
    i = 1
    while i < len(fields):
        
        penalty_type = fields[i]
        penalty_info = fields[i+1]
        
        # Mismatch
        if penalty_type is "*":
            mismatches += 1
        # Insertion
        elif penalty_type is "+":
            insertions += 1
        # Deletion
        elif penalty_type is "-":
            deletions += 1
        # Match
        elif penalty_type == ":":
            matches += int(penalty_info)
            #print (penalty_info)
        
        i += 2
        
    return mismatches, insertions, deletions, matches
    

In [3]:
def summarize_penalties(filename):

    inputfile = open(filename, "r")

    total_bases = 0
    num_mismatches = 0
    num_insertions = 0
    num_deletions = 0
    num_matches = 0

    for line in inputfile:

        # Header lines start with '@'
        if line[0] is "@":
            continue

        fields = line.split()
        seq = fields[9]

        # Find cs string, if present
        # If not present, the read was not mapped
        cs_string = "None"
        i = 10
        while i < len(fields):
            entry = fields[i]
            if "cs:Z:" in entry: 
                cs_string = entry
                break
            i += 1

        # If read is mapped, count bases and penalties
        if cs_string is not "None":
            total_bases += len(seq)

            accuracy_info = cs_string.split("cs:Z:")[1]

            mis, ins, delet, match = count_penalties(accuracy_info)

            num_mismatches += mis
            num_insertions += ins
            num_deletions += delet
            num_matches += match



    # Error Rates
    print ("Mismatch Rate: " + str(float(num_mismatches)/float(total_bases)))
    print ("Insertion Rate: " + str(float(num_insertions)/float(total_bases)))
    print ("Deletion Rate: " + str(float(num_deletions)/float(total_bases)))

    # This considers the length of insertions and deletions
    print ()
    print ("Num Matches: " + str(num_matches))
    print ("Total Bases: " + str(total_bases))
    print ("Match Rate: " + str(float(num_matches)/float(total_bases)))

    # This considers just the amount of insertions and deletions
    all_types = num_deletions + num_insertions + num_mismatches + num_matches
    match_rate_v2 = num_matches / float(all_types)
    print ()
    print ("Match Rate V2: " + str(match_rate_v2))
    
    return (all_types, num_matches, num_mismatches)


In [7]:
def generateSummary(chrval):
   
    filename = "resnetcard4_mapped_results/" + chrval + "_resnetcard4.sam"
    numMapped = printNumMapped(filename)
    numBases, numMatches, numMismatches = summarize_penalties(filename)

    print (numMapped)
    return (numMapped, numBases, numMatches, numMismatches)


In [8]:
mappedSum = 0
totalBases = 0
totalMatches = 0
totalMismatches = 0

mapped, bases, matched, mismatched = generateSummary("chr1")
mappedSum += mapped; totalBases += bases; totalMatches += matched; totalMismatches += mismatched

generateSummary("chr2")
mappedSum += mapped; totalBases += bases; totalMatches += matched; totalMismatches += mismatched
generateSummary("chr3")
mappedSum += mapped; totalBases += bases; totalMatches += matched; totalMismatches += mismatched
generateSummary("chr4")
mappedSum += mapped; totalBases += bases; totalMatches += matched; totalMismatches += mismatched



print ("Final Sums:")
print (mappedSum)
print (totalBases)
print (totalMatches)
print (totalMismatches)

resnetcard4_mapped_results/chr1_resnetcard4.sam
3131
193
193

Mismatch Rate: 0.053641847623660026
Insertion Rate: 0.009125892385656477
Deletion Rate: 0.0562325741065446

Num Matches: 489662
Total Bases: 1028283
Match Rate: 0.47619381045879394

Match Rate V2: 0.8000647029220885
193
resnetcard4_mapped_results/chr2_resnetcard4.sam
3102
258
258

Mismatch Rate: 0.05240216262893389
Insertion Rate: 0.00905275455812462
Deletion Rate: 0.05233946162857515

Num Matches: 564953
Total Bases: 1259948
Match Rate: 0.4483939019705575

Match Rate V2: 0.7975867112411199
258
resnetcard4_mapped_results/chr3_resnetcard4.sam
3122
199
199

Mismatch Rate: 0.05224371568949513
Insertion Rate: 0.009659081246022376
Deletion Rate: 0.053120023773996654

Num Matches: 510302
Total Bases: 1117187
Match Rate: 0.45677402261215

Match Rate V2: 0.7988397067018992
199
resnetcard4_mapped_results/chr4_resnetcard4.sam
3136
174
174

Mismatch Rate: 0.04866614665353847
Insertion Rate: 0.008971701908539987
Deletion Rate: 0.0503727

In [9]:
#Match Rate
1958648 / 2448112

0.8000647029220885

In [10]:
#Mismatch Rate
220636 / 2448112

0.09012496160306391

In [11]:
# Indel Rate
(2448112 - 220636 - 1958648) / 2448112

0.10981033547484756